# 6 WorkFlow

### 6.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 6.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [1]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 6.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Wed Nov 26 03:02:15 AM 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660365,35.3,1454491,77.7,1454491,77.7
Vcells,1226539,9.4,8388608,64.0,1975136,15.1


In [3]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings




#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [4]:
PARAM <- list()
PARAM$semilla_primigenia <- 200029

PARAM$experimento <- 6300
PARAM$dataset <- "gerencial_competencia_2025.csv"

#### Carpeta del Experimento

In [5]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

ERROR: Error in setwd("/content/buckets/b1/exp"): cannot change working directory


### 6.3.1   Preprocesamiento del dataset

#### 6.3.1.1  DT incorporar dataset

In [6]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

ERROR: Error in fread(paste0("/content/datasets/", PARAM$dataset)): File '/content/datasets/gerencial_competencia_2025.csv' does not exist or is non-readable. getwd()=='/content'


In [ ]:
head(dataset)

In [ ]:
# Ejecuta un pequeño script JavaScript desde R para mantener activa la sesión
IRdisplay::display_javascript("
function clickConnect() {
  console.log('Manteniendo sesión activa desde R...');
  const btn = document.querySelector('#top-toolbar > colab-connect-button');
  if (btn && btn.shadowRoot) {
    const connect = btn.shadowRoot.querySelector('#connect');
    if (connect) connect.click();
  }
}
setInterval(clickConnect, 60000);
")

#### 6.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [ ]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 6.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [10]:
#PASO 1
library(readxl)
library(data.table)
library(dplyr)
fapce <- read_excel("/content/datasets/Indice-FACPCE.xlsx") %>%
  mutate(
    # ---- FOTO_MES CORREGIDO ----
    # Eliminar el último caracter: "2020050" → "202005"
    foto_mes = gsub("[^0-9]", "", foto_mes),
    foto_mes = substr(foto_mes, 1, 6),

    # ---- IPC ----
    IPC = gsub("[^0-9]", "", IPC),
    IPC = as.numeric(IPC),

    # ---- coeficiente ajustes ----
    coeficiente_de_ajuste = gsub(",", ".", coeficiente_de_ajuste),
    coeficiente_de_ajuste = gsub("[^0-9.]", "", coeficiente_de_ajuste),
    coeficiente_de_ajuste = as.numeric(coeficiente_de_ajuste)
  ) %>%
  as.data.table()





ERROR: Error: `path` does not exist: ‘/content/datasets/Indice-FACPCE.xlsx’


In [13]:
head(fapce)

foto_mes,IPC,coeficiente_de_ajuste
<chr>,<dbl>,<dbl>
202005,3149,1.678311
202006,322,1.641304
202007,3282,1.610299
202008,3371,1.567784
202009,3466,1.524812
202010,3597,1.469280


In [14]:
#PASO 2
dataset <- as.data.table(dataset)

# asegurar foto_mes como texto tipo "202005"
dataset[, foto_mes := as.character(foto_mes)]
dataset[, foto_mes := gsub("[^0-9]", "", foto_mes)]
dataset[, foto_mes := substr(foto_mes, 1, 6)]





In [15]:
#PASO 3
cols_borrar <- c("coeficiente_de_ajuste", "coeficiente_de_ajuste.x", "coeficiente_de_ajuste.y")

for (col in cols_borrar) {
  if (col %in% names(dataset)) dataset[, (col) := NULL]
}





In [16]:
#PASO 4
dataset <- merge(
  dataset,
  fapce[, .(foto_mes, coeficiente_de_ajuste)],
  by = "foto_mes",
  all.x = TRUE
)



In [17]:
#PASO 5 MERGE POR FOTO_MES
vars_ajustar <- c(
  "mrentabilidad",
  "mrentabilidad_annual",
  "mcomisiones",
  "mactivos_margen",
  "mpasivos_margen",
  "mcuenta_corriente",
  "mcaja_ahorro",
  "mcuentas_saldo",
  "mtarjeta_visa_consumo",
  "mtarjeta_master_consumo",
  "mprestamos_personales",
  "mpayroll",
  "Master_mpagominimo",
  "Visa_mpagominimo"
)

dataset[, (vars_ajustar) := lapply(.SD, as.numeric), .SDcols = vars_ajustar]



In [18]:
#PASO 6
dataset[
  ,
  (vars_ajustar) := lapply(.SD, function(x) x * coeficiente_de_ajuste),
  .SDcols = vars_ajustar
]



In [19]:
head(dataset)

foto_mes,numero_de_cliente,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,chomebanking_transacciones,ctrx_quarter,Master_status,Master_fechaalta,Master_mpagominimo,Visa_status,Visa_fechaalta,Visa_mpagominimo,clase_ternaria,coeficiente_de_ajuste
<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>,<dbl>
202005,29186441,1,61,282,3952.3207,73445.539,2823.1534,-233.31874,1195.9641,⋯,20,47,0,6507,0.0000,0,7671,1752.10589,CONTINUA,1.678311
202005,29187961,1,61,312,8502.8416,79330.938,6504.4940,-5934.55654,7638.1760,⋯,228,232,0,5939,3386.0923,0,8090,12579.72656,CONTINUA,1.678311
202005,29193101,1,67,365,8071.7004,63770.196,11719.1896,-4971.55872,1862.0185,⋯,133,149,0,2026,4764.1531,0,2026,2775.80821,CONTINUA,1.678311
202005,29193281,1,54,158,5000.5599,148424.064,5622.3572,-1524.82907,994.7347,⋯,10,115,0,3125,1417.4340,0,3350,3405.77887,CONTINUA,1.678311
202005,29198891,1,54,312,5049.5330,37874.972,459.3200,-1080.61383,4993.8467,⋯,124,32,0,7202,393.7317,0,7242,2342.70338,CONTINUA,1.678311
202005,29205441,1,72,6,661.5732,2698.606,267.6738,-16.64884,352.4620,⋯,41,32,0,157,0.0000,0,157,98.43292,CONTINUA,1.678311


In [20]:
# sin codigo en esta primera version del workflow

#### 6.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [22]:
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  dataset[, kmes := as.numeric(foto_mes) %% 100]

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]


In [23]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "foto_mes"                    "numero_de_cliente"          
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"             
[33] "coeficiente_de_ajuste"       "kmes"                       
[35] "mpayroll_sobre_edad"

In [24]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

Loading required package: lightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lightgbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: lightgbm



In [26]:
AgregaVarRandomForest <- function() {

  cat("inicio AgregaVarRandomForest()\n")
  gc(verbose = FALSE)

  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$train$clase01_valor1,
           clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c("clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
            as.integer(foto_mes %in% PARAM$train$training)]

  dtrain <- lgb.Dataset(
    data  = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
    data = dtrain,
    param = PARAM$lgb_param,
    verbose = -100
  )

  cat("Fin construccion RandomForest\n")

  lgb.save(modelo, file = "modelo.model")

  qarbolitos <- copy(PARAM$lgb_param$num_iterations)

  periodos <- dataset[, unique(foto_mes)]

  for (periodo in periodos) {
    cat("periodo = ", periodo, "\n")
    datamatrix <- data.matrix(dataset[foto_mes == periodo, campos_buenos, with = FALSE])

    cat("Inicio prediccion\n")
    prediccion <- predict(
      modelo,
      datamatrix,
      type = "leaf"
    )
    cat("Fin prediccion\n")

    for (arbolito in 1:qarbolitos) {
      cat(arbolito, " ")
      hojas_arbol <- unique(prediccion[, arbolito])

      for (pos in seq_along(hojas_arbol)) {

        nodo_id <- hojas_arbol[pos]

        dataset[foto_mes == periodo,
                paste0("rf_",
                       sprintf("%03d", arbolito),
                       "_",
                       sprintf("%03d", nodo_id)
                ) :=
                  as.integer(nodo_id == prediccion[, arbolito])]
      }

      rm(hojas_arbol)
    }
    cat("\n")

    rm(prediccion)
    rm(datamatrix)
    gc(verbose = FALSE)
  }

  gc(verbose = FALSE)

  dataset[, clase01 := NULL ]
}


In [30]:
# ======================================================================
# PARAMETROS DE FEATURE ENGINEERING PARA HOJAS DE RANDOM FOREST
# ======================================================================

PARAM <- list()
PARAM$train <- list()

# Estos CUATRO parametros son los que se deben modificar
PARAM$arbolitos       <- 20
PARAM$hojas_por_arbol <- 16
PARAM$datos_por_hoja  <- 100
PARAM$mtry_ratio      <- 0.2

# Estos son quasi fijos
PARAM$train$clase01_valor1 <- c("BAJA+2", "BAJA+1")
PARAM$train$training        <- c(202101, 202102, 202103)

# Estos TAMBIEN son quasi fijos
PARAM$lgb_param <- list(
    # parametros que se pueden cambiar
    num_iterations          = PARAM$arbolitos,
    num_leaves              = PARAM$hojas_por_arbol,
    min_data_in_leaf        = PARAM$datos_por_hoja,
    feature_fraction_bynode = PARAM$mtry_ratio,

    # para que LightGBM emule Random Forest
    boosting         = "rf",
    bagging_fraction = (1.0 - 1.0 / exp(1.0)),
    bagging_freq     = 1.0,
    feature_fraction = 1.0,

    # genericos de LightGBM
    max_bin                 = 31L,
    objective               = "binary",
    first_metric_only       = TRUE,
    boost_from_average      = TRUE,
    feature_pre_filter      = FALSE,
    force_row_wise          = TRUE,
    verbosity               = -100,
    max_depth               = -1L,
    min_gain_to_split       = 0.0,
    min_sum_hessian_in_leaf = 0.001,
    lambda_l1               = 0.0,
    lambda_l2               = 0.0,

    pos_bagging_fraction = 1.0,
    neg_bagging_fraction = 1.0,
    is_unbalance         = FALSE,
    scale_pos_weight     = 1.0,

    drop_rate  = 0.1,
    max_drop   = 50,
    skip_drop  = 0.5,

    extra_trees = FALSE
)

# ======================================================================
# FUNCION PRINCIPAL
# ======================================================================

AgregaVarRandomForest <- function() {

  cat("inicio AgregaVarRandomForest()\n")
  gc(verbose = FALSE)

  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$train$clase01_valor1,
           clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c("clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
            as.integer(foto_mes %in% PARAM$train$training)]

  dtrain <- lgb.Dataset(
    data  = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
    data    = dtrain,
    param   = PARAM$lgb_param,
    verbose = -100
  )

  cat("Fin construccion RandomForest\n")

  lgb.save(modelo, file = "modelo.model")

  qarbolitos <- copy(PARAM$lgb_param$num_iterations)

  periodos <- dataset[, unique(foto_mes)]

  for (periodo in periodos) {
    cat("periodo = ", periodo, "\n")
    datamatrix <- data.matrix(dataset[foto_mes == periodo, campos_buenos, with = FALSE])

    cat("Inicio prediccion\n")
    prediccion <- predict(
      modelo,
      datamatrix,
      type = "leaf"
    )
    cat("Fin prediccion\n")

    for (arbolito in 1:qarbolitos) {
      cat(arbolito, " ")
      hojas_arbol <- unique(prediccion[, arbolito])

      for (pos in seq_along(hojas_arbol)) {

        nodo_id <- hojas_arbol[pos]

        dataset[foto_mes == periodo,
                paste0("rf_",
                       sprintf("%03d", arbolito),
                       "_",
                       sprintf("%03d", nodo_id)
                ) :=
                  as.integer(nodo_id == prediccion[, arbolito])]
      }

      rm(hojas_arbol)
    }
    cat("\n")

    rm(prediccion)
    rm(datamatrix)
    gc(verbose = FALSE)
  }

  gc(verbose = FALSE)

  dataset[, clase01 := NULL ]
}


In [31]:
# Feature Engineering agregando variables de Random Forest
AgregaVarRandomForest()

inicio AgregaVarRandomForest()
Fin construccion RandomForest
periodo =  202005 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202006 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202007 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202008 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202009 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202010 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202011 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202012 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7

#### 6.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [ ]:
# No se implementa Feature Engineering a partir de Random Forest

#### 6.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [32]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

cols_promedio <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria","cliente_edad", "cliente_antiguedad", "internet", "cdescubierto_preacordado", "thomebanking",
     "Master_fultimo_cierre", "Master_fechaalta", "Master_status", "Visa_status", "Visa_fultimo_cierre", "Visa_fechaalta")
) )



# Probando EMA

# función general para EMA(N)
emaN <- function(x, n) {
  alpha <- 2 / (n + 1)

  # si todos NA -> devolver NA
  if (all(is.na(x))) return(rep(NA_real_, length(x)))

  # primer EMA = primer valor no NA
  first_idx <- which(!is.na(x))[1]
  ema_vec <- rep(NA_real_, length(x))
  ema_vec[first_idx] <- x[first_idx]

  # recursión
  if (first_idx < length(x)) {
    for (i in (first_idx+1):length(x)) {
      ema_vec[i] <- alpha * x[i] + (1 - alpha) * ema_vec[i - 1]
    }
  }

  return(ema_vec)
}


# aplicar EMA3, EMA6 y EMA12 a todas las columnas de cols_promedio
dataset[
  , c(
      paste0(cols_promedio, "_ema3"),
      paste0(cols_promedio, "_ema6"),
      paste0(cols_promedio, "_ema12")
    ) := c(
      lapply(.SD, function(col) emaN(col, 3)),
      lapply(.SD, function(col) emaN(col, 6)),
      lapply(.SD, function(col) emaN(col, 12))
    ),
  by = numero_de_cliente,
  .SDcols = cols_promedio
]

# creo ratio ult 12 meses vs 6 meses, ult 6 vs ult 3 y actuales vs 3
for (vcol in cols_promedio)
{
    ema6 <- paste0(vcol, "_ema6")
    ema3 <- paste0(vcol, "_ema3")
    ema12 <- paste0(vcol, "_ema12")

    # Ratio EMA3 / EMA6  (aceleración)
    dataset[, paste0(vcol, "ema3_ema6_ratio") :=
              fifelse(get(ema6) == 0 | is.na(get(ema6)),
                      NA_real_, get(ema3) / get(ema6))]

    # Ratio actual / EMA6 - Cambios a mediano plazo
    dataset[, paste0(vcol, "actual_ema6_ratio") :=
              fifelse(get(ema6) == 0 | is.na(get(ema6)),
                      NA_real_, get(vcol) / get(ema6))]


    # Ratio actual / EMA3 - Cambios a corto plazo
    dataset[, paste0(vcol, "actual_ema3_ratio") :=
              fifelse(get(ema3) == 0 | is.na(get(ema3)),
                      NA_real_, get(vcol) / get(ema3))]

    # Ratio actual / EMA12 - Cambios a largo plazo
        dataset[, paste0(vcol, "actual_ema12_ratio") :=
              fifelse(get(ema12) == 0 | is.na(get(ema12)),
                      NA_real_, get(vcol) / get(ema12))]
}

#Elimino emas y dejo solo los ratios
cols_a_borrar <- grep("(_ema3|_ema6|_ema12)$", names(dataset), value = TRUE)
dataset[, (cols_a_borrar) := NULL]

In [33]:
#VERIFICACIÓN DE CAMPOS RECIÉN CREADOS
ncol(dataset)
colnames(dataset)


[1] 1731

[1] "foto_mes"                                     
   [2] "numero_de_cliente"                            
   [3] "internet"                                     
   [4] "cliente_edad"                                 
   [5] "cliente_antiguedad"                           
   [6] "mrentabilidad"                                
   [7] "mrentabilidad_annual"                         
   [8] "mcomisiones"                                  
   [9] "mactivos_margen"                              
  [10] "mpasivos_margen"                              
  [11] "cproductos"                                   
  [12] "mcuenta_corriente"                            
  [13] "mcaja_ahorro"                                 
  [14] "cdescubierto_preacordado"                     
  [15] "mcuentas_saldo"                               
  [16] "ctarjeta_visa_transacciones"                  
  [17] "mtarjeta_visa_consumo"                        
  [18] "mtarjeta_master_consumo"                      
  [19] "mprestamos_personales"                        
  [20] "cpayroll_trx"                                 
  [21] "mpayroll"                                     
  [22] "ccomisiones_mantenimiento"                    
  [23] "ccallcenter_transacciones"                    
  [24] "chomebanking_transacciones"                   
  [25] "ctrx_quarter"                                 
  [26] "Master_status"                                
  [27] "Master_fechaalta"                             
  [28] "Master_mpagominimo"                           
  [29] "Visa_status"                                  
  [30] "Visa_fechaalta"                               
  [31] "Visa_mpagominimo"                             
  [32] "clase_ternaria"                               
  [33] "coeficiente_de_ajuste"                        
  [34] "kmes"                                         
  [35] "mpayroll_sobre_edad"                          
  [36] "entrenamiento"                                
  [37] "rf_001_008"                                   
  [38] "rf_001_001"                                   
  [39] "rf_001_011"                                   
  [40] "rf_001_009"                                   
  [41] "rf_001_015"                                   
  [42] "rf_001_002"                                   
  [43] "rf_001_007"                                   
  [44] "rf_001_013"                                   
  [45] "rf_001_004"                                   
  [46] "rf_001_014"                                   
  [47] "rf_001_003"                                   
  [48] "rf_001_006"                                   
  [49] "rf_001_000"                                   
  [50] "rf_001_005"                                   
  [51] "rf_001_010"                                   
  [52] "rf_002_011"                                   
  [53] "rf_002_013"                                   
  [54] "rf_002_014"                                   
  [55] "rf_002_007"                                   
  [56] "rf_002_001"                                   
  [57] "rf_002_008"                                   
  [58] "rf_002_003"                                   
  [59] "rf_002_010"                                   
  [60] "rf_002_015"                                   
  [61] "rf_002_002"                                   
  [62] "rf_002_009"                                   
  [63] "rf_002_012"                                   
  [64] "rf_002_006"                                   
  [65] "rf_002_005"                                   
  [66] "rf_002_000"                                   
  [67] "rf_002_004"                                   
  [68] "rf_003_013"                                   
  [69] "rf_003_012"                                   
  [70] "rf_003_015"                                   
  [71] "rf_003_003"                                   
  [72] "rf_003_002"                                   
  [73] "rf_003_001"                        

Verificacion de los campos recien creados

In [34]:
ncol(dataset)
colnames(dataset)

[1] 1731

[1] "foto_mes"                                     
   [2] "numero_de_cliente"                            
   [3] "internet"                                     
   [4] "cliente_edad"                                 
   [5] "cliente_antiguedad"                           
   [6] "mrentabilidad"                                
   [7] "mrentabilidad_annual"                         
   [8] "mcomisiones"                                  
   [9] "mactivos_margen"                              
  [10] "mpasivos_margen"                              
  [11] "cproductos"                                   
  [12] "mcuenta_corriente"                            
  [13] "mcaja_ahorro"                                 
  [14] "cdescubierto_preacordado"                     
  [15] "mcuentas_saldo"                               
  [16] "ctarjeta_visa_transacciones"                  
  [17] "mtarjeta_visa_consumo"                        
  [18] "mtarjeta_master_consumo"                      
  [19] "mprestamos_personales"                        
  [20] "cpayroll_trx"                                 
  [21] "mpayroll"                                     
  [22] "ccomisiones_mantenimiento"                    
  [23] "ccallcenter_transacciones"                    
  [24] "chomebanking_transacciones"                   
  [25] "ctrx_quarter"                                 
  [26] "Master_status"                                
  [27] "Master_fechaalta"                             
  [28] "Master_mpagominimo"                           
  [29] "Visa_status"                                  
  [30] "Visa_fechaalta"                               
  [31] "Visa_mpagominimo"                             
  [32] "clase_ternaria"                               
  [33] "coeficiente_de_ajuste"                        
  [34] "kmes"                                         
  [35] "mpayroll_sobre_edad"                          
  [36] "entrenamiento"                                
  [37] "rf_001_008"                                   
  [38] "rf_001_001"                                   
  [39] "rf_001_011"                                   
  [40] "rf_001_009"                                   
  [41] "rf_001_015"                                   
  [42] "rf_001_002"                                   
  [43] "rf_001_007"                                   
  [44] "rf_001_013"                                   
  [45] "rf_001_004"                                   
  [46] "rf_001_014"                                   
  [47] "rf_001_003"                                   
  [48] "rf_001_006"                                   
  [49] "rf_001_000"                                   
  [50] "rf_001_005"                                   
  [51] "rf_001_010"                                   
  [52] "rf_002_011"                                   
  [53] "rf_002_013"                                   
  [54] "rf_002_014"                                   
  [55] "rf_002_007"                                   
  [56] "rf_002_001"                                   
  [57] "rf_002_008"                                   
  [58] "rf_002_003"                                   
  [59] "rf_002_010"                                   
  [60] "rf_002_015"                                   
  [61] "rf_002_002"                                   
  [62] "rf_002_009"                                   
  [63] "rf_002_012"                                   
  [64] "rf_002_006"                                   
  [65] "rf_002_005"                                   
  [66] "rf_002_000"                                   
  [67] "rf_002_004"                                   
  [68] "rf_003_013"                                   
  [69] "rf_003_012"                                   
  [70] "rf_003_015"                                   
  [71] "rf_003_003"                                   
  [72] "rf_003_002"                                   
  [73] "rf_003_001"                        

#### 6.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [ ]:
# No se implementa la reduccion de la dimensionalidad con canaritos

### 6.3.2 Modelado

#### 6.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  SIN undersampling

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [35]:
# =====================================================
# CONFIGURACIÓN DEL EXPERIMENTO - TS Training Strategy #16
# Campos clave en datos de entrenamiento: ¿Mantenerlos o hacerlos nulos?
# =====================================================

# Seleccione UNO de los siguientes experimentos descomentando la línea correspondiente:
# (Asegúrese de que solo UNA línea esté descomentada)

# Experimento 1: Modelo Base - Incluye todas las variables
#  PARAM$excluir_campos <- c()  # <- ACTIVO POR DEFECTO

# Experimento 2: Exclusión de numero_de_cliente
# PARAM$excluir_campos <- c("numero_de_cliente")

# Experimento 3: Exclusión de foto_mes
# PARAM$excluir_campos <- c("foto_mes")

# Experimento 4: Exclusión de ambas variables
 PARAM$excluir_campos <- c("numero_de_cliente", "foto_mes")

# Imprimir configuración activa
cat("\n========== CONFIGURACIÓN ACTIVA ==========\n")
if (length(PARAM$excluir_campos) == 0) {
    cat("Experimento 1: Modelo Base (todas las variables incluidas)\n")
} else if (identical(PARAM$excluir_campos, c("numero_de_cliente"))) {
    cat("Experimento 2: Exclusión de numero_de_cliente\n")
} else if (identical(PARAM$excluir_campos, c("foto_mes"))) {
    cat("Experimento 3: Exclusión de foto_mes\n")
} else if (setequal(PARAM$excluir_campos, c("numero_de_cliente", "foto_mes"))) {
    cat("Experimento 4: Exclusión de numero_de_cliente y foto_mes\n")
}
cat("Campos a excluir:", ifelse(length(PARAM$excluir_campos) == 0, "Ninguno", paste(PARAM$excluir_campos, collapse=", ")), "\n")
cat("==========================================\n\n")


========== CONFIGURACIÓN ACTIVA ==========
Experimento 4: Exclusión de numero_de_cliente y foto_mes
Campos a excluir: numero_de_cliente, foto_mes 



In [36]:
# los campos en los que se entrena
# Primero obtenemos todos los campos menos los que siempre se excluyen
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

# Ahora excluimos los campos según el experimento configurado
if (exists("PARAM") && !is.null(PARAM$excluir_campos) && length(PARAM$excluir_campos) > 0) {
    cat("Excluyendo campos del entrenamiento:", paste(PARAM$excluir_campos, collapse=", "), "\n")
    campos_buenos <- setdiff(campos_buenos, PARAM$excluir_campos)
    cat("Total de campos después de exclusión:", length(campos_buenos), "\n\n")
} else {
    cat("No se excluyen campos adicionales del entrenamiento\n")
    cat("Total de campos:", length(campos_buenos), "\n\n")
}

Excluyendo campos del entrenamiento: numero_de_cliente, foto_mes 
Total de campos después de exclusión: 1728 



In [42]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

ERROR: Error in `[.data.table`(dataset, , `:=`(fold_train, foto_mes %in% PARAM$trainingstrategy$training & : RHS of assignment to existing column 'fold_train' is zero length but not NULL. If you intend to delete the column use NULL. Otherwise, the RHS must have length > 0; e.g., NA_integer_. If you are trying to change the column type to be an empty list column then, as with all column type changes, provide a full length RHS vector such as vector('list',nrow(DT)); i.e., 'plonk' in the new column.


In [41]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

ERROR: Error in `[.data.table`(dataset, foto_mes %in% PARAM$trainingstrategy$validate, : j (the 2nd argument inside [...]) is a single symbol but column name 'clase01' is not found. If you intended to select columns using a variable in calling scope, please try DT[, ..clase01]. The .. prefix conveys one-level-up similar to a file system path.


####  6.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [ ]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [ ]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM
num_interations <- 10

# parametros fijos del LightGBM
PARAM
param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,
  learning_rate= 0.03,
  feature_fraction= 0.5,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)

PARAM
hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
  makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L)
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [ ]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [ ]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [ ]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [ ]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

print(PARAM$out$lgbm$mejores_hiperparametros)

### 6.3.3 Produccion

#### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [ ]:
PARAM$trainingstrategy$final_train <- c( 202107,
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

##### Final Training Hyperparameters

In [ ]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [ ]:
final_model <- lgb.train(
  data= dfinal_train,
  param= param_final,
  verbose= -100
)

In [ ]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...

lgb.save(final_model, "modelo.txt")

In [ ]:
# ahora imprimo la importancia de variables

tb_importancia <- as.data.table(lgb.importance(final_model))
archivo_importancia <- "impo.txt"

fwrite( tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

#### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [ ]:
# aplico final_model   a dfuture

prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with= FALSE])
)

##### Tabla Prediccion

In [ ]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

#### Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
<br>El notebook esta preparado para la Modalidad Gerencial, los analistas deben hacer cambios.
<br> Los analistas deben cambiar **competencia** a SU competencia  "data-mining-analista-jr-2025-a"   o  la original "data-mining-analista-sr-2025-a"  para los Senior
<br> Los cortes  dependen de la cantidad de registros, multiplicar por 2 para los Analistas Jr y por 10 para los Analista Sr

Los Analista Sr luego de meditar cuidadosamente reducirán la cantidad de cortes

In [ ]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "labo-i-2025-virtual-gerencial"
PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

In [ ]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")